<a href="https://colab.research.google.com/github/gkunal8019/ai_agent_for_system/blob/main/agent_ai_command.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain
!pip install openai
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2

In [3]:
import openai
import os

os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.environ["OPENAI_API_KEY"]

In [6]:
import subprocess
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

def run_command(command):
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    return result.stdout

def extract_gpu_memory(nvidia_smi_output):
    lines = nvidia_smi_output.split("\n")
    for line in lines:
        if "MiB" in line:
            memory_info = line.split("|")[2].strip()
            used_memory, total_memory = memory_info.split("/")
            total_memory = total_memory.split()[0].strip()
            return int(total_memory)
    return None

def recommend_gpt_size(gpu_memory, user_input):
    if user_input.lower() == "large":
        if gpu_memory >= 8192:
            return "Based on your GPU's available memory of {} MB, you can use a 'large' GPT model size.".format(gpu_memory)
        else:
            return "Based on your GPU's available memory of {} MB, I recommend using a smaller GPT model size to ensure optimal performance. A 'medium' or 'small' model would be more suitable for your GPU.".format(gpu_memory)
    elif user_input.lower() == "medium":
        if gpu_memory >= 4096:
            return "Based on your GPU's available memory of {} MB, you can use a 'medium' GPT model size.".format(gpu_memory)
        else:
            return "Based on your GPU's available memory of {} MB, I recommend using a 'small' GPT model size for optimal performance.".format(gpu_memory)
    else:
        return "Based on your GPU's available memory of {} MB, a 'small' GPT model size would be the most suitable choice.".format(gpu_memory)

tools = [
    Tool(
        name="GPU Memory",
        func=run_command,
        description="Runs the nvidia-smi command to get GPU memory information",
    ),
]

llm = OpenAI(temperature=0)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

user_input = input("What size GPT model do you want to use? (small, medium, large) ")

agent_response = agent.run("To determine the appropriate GPT model size, we need to know the available GPU memory. Run the nvidia-smi command to get this information.")

gpu_memory = extract_gpu_memory(agent_response)

if gpu_memory:
    recommendation = recommend_gpt_size(gpu_memory, user_input)
    print("Recommendation:", recommendation)
else:
    print("Unable to determine GPU memory. Please make sure you have an NVIDIA GPU and the nvidia-smi command is available.")

<ipython-input-6-58175dc99be4>:42: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)
<ipython-input-6-58175dc99be4>:44: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(


What size GPT model do you want to use? (small, medium, large) what is gpt size


<ipython-input-6-58175dc99be4>:53: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_response = agent.run("To determine the appropriate GPT model size, we need to know the available GPU memory. Run the nvidia-smi command to get this information.")




> Entering new AgentExecutor chain...
 We should always check the available GPU memory before selecting a GPT model size.
Action: GPU Memory
Action Input: nvidia-smi
Observation: 
Thought: We can see the available GPU memory in the output of the nvidia-smi command.
Action: GPU Memory
Action Input: nvidia-smi
Observation: 
Thought: We can use the available GPU memory to determine the maximum GPT model size that can be used.
Action: GPU Memory
Action Input: nvidia-smi
Observation: 
Thought: We should also consider the memory requirements of other processes running on the GPU.
Action: GPU Memory
Action Input: nvidia-smi
Observation: 
Thought: We can now confidently select the appropriate GPT model size based on the available GPU memory and other memory requirements.
Final Answer: The appropriate GPT model size can be determined by checking the available GPU memory and considering other memory requirements.

> Finished chain.
Unable to determine GPU memory. Please make sure you have an N

In [5]:
!!pip install langchain-community

['Collecting langchain-community',
 '  Downloading langchain_community-0.3.2-py3-none-any.whl.metadata (2.8 kB)',
 'Requirement already satisfied: PyYAML>=5.3 in /usr/local/lib/python3.10/dist-packages (from langchain-community) (6.0.2)',
 'Requirement already satisfied: SQLAlchemy<3,>=1.4 in /usr/local/lib/python3.10/dist-packages (from langchain-community) (2.0.35)',
 'Requirement already satisfied: aiohttp<4.0.0,>=3.8.3 in /usr/local/lib/python3.10/dist-packages (from langchain-community) (3.10.9)',
 'Collecting dataclasses-json<0.7,>=0.5.7 (from langchain-community)',
 '  Downloading dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)',
 'Requirement already satisfied: langchain<0.4.0,>=0.3.3 in /usr/local/lib/python3.10/dist-packages (from langchain-community) (0.3.3)',
 'Requirement already satisfied: langchain-core<0.4.0,>=0.3.10 in /usr/local/lib/python3.10/dist-packages (from langchain-community) (0.3.10)',
 'Requirement already satisfied: langsmith<0.2.0,>=0.1.125 in /us

working

In [24]:
# import subprocess
# import os
# from langchain.llms import OpenAI
# from langchain.agents import initialize_agent, Tool
# from langchain.agents import AgentType

# def run_command(command):
#     try:
#         result = subprocess.run(command, shell=True, check=True, capture_output=True, text=True)
#         return result.stdout
#     except subprocess.CalledProcessError as e:
#         return f"Error executing command: {e.stderr}"

# def extract_gpu_memory(nvidia_smi_output):
#     lines = nvidia_smi_output.split("\n")
#     for line in lines:
#         if "MiB" in line:
#             memory_info = line.split("|")[2].strip()
#             used_memory, total_memory = memory_info.split("/")
#             total_memory = total_memory.split()[0].strip()
#             return int(total_memory)
#     return None

# def recommend_gpt_size(gpu_memory, user_input):
#     if user_input.lower() == "large":
#         if gpu_memory >= 8192:
#             return f"Based on your GPU's available memory of {gpu_memory} MB, you can use a 'large' GPT model size."
#         else:
#             return f"Based on your GPU's available memory of {gpu_memory} MB, I recommend using a smaller GPT model size to ensure optimal performance. A 'medium' or 'small' model would be more suitable for your GPU."
#     elif user_input.lower() == "medium":
#         if gpu_memory >= 4096:
#             return f"Based on your GPU's available memory of {gpu_memory} MB, you can use a 'medium' GPT model size."
#         else:
#             return f"Based on your GPU's available memory of {gpu_memory} MB, I recommend using a 'small' GPT model size for optimal performance."
#     else:
#         return f"Based on your GPU's available memory of {gpu_memory} MB, a 'small' GPT model size would be the most suitable choice."

# def llm_query(query):
#     response = agent.run(query)
#     return response

# def count_files_in_directory(directory):
#     total_files = 0
#     for root, dirs, files in os.walk(directory):
#         total_files += len(files)
#     return total_files

# tools = [
#     Tool(
#         name="CPU Temperature",
#         func=lambda x: run_command("sensors"),
#         description="Runs the sensors command to get CPU temperature information",
#     ),
#     Tool(
#         name="Disk Space",
#         func=lambda x: run_command("df -h"),
#         description="Runs the df -h command to get disk space information",
#     ),
#     Tool(
#         name="Memory Usage",
#         func=lambda x: run_command("free -h"),
#         description="Runs the free -h command to get memory usage information",
#     ),
# ]

# llm = OpenAI(temperature=0)

# agent = initialize_agent(
#     tools,
#     llm,
#     agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#     verbose=True
# )

# def main():
#     while True:
#         user_input = input("Enter your query (or 'exit' to quit): ").strip().lower()

#         if user_input == 'exit':
#             break

#         if "how many files" in user_input:
#             directory = input("Enter the directory path: ")
#             total_files = count_files_in_directory(directory)
#             print(f"Total files in {directory}: {total_files}")
#         elif any(keyword in user_input for keyword in ["gpu", "nvidia", "gpt model size"]):
#             print("Executing nvidia-smi command:")
#             gpu_info = run_command("nvidia-smi")
#             print(gpu_info)
#             gpu_memory = extract_gpu_memory(gpu_info)
#             if gpu_memory:
#                 print(f"Total GPU Memory: {gpu_memory} MB")
#                 if "gpt model size" in user_input:
#                     size_input = input("What size GPT model do you want to use? (small, medium, large) ").strip().lower()
#                     recommendation = recommend_gpt_size(gpu_memory, size_input)
#                     print("Recommendation:", recommendation)
#             else:
#                 print("Unable to determine GPU memory. Please make sure you have an NVIDIA GPU and the nvidia-smi command is available.")
#         elif any(tool.name.lower() in user_input for tool in tools):
#             for tool in tools:
#                 if tool.name.lower() in user_input:
#                     print(f"Executing command: {tool.name}")
#                     print(tool.func(None))
#                     break
#         else:
#             print(llm_query(user_input))

# if __name__ == "__main__":
#     main()

In [23]:
import subprocess
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

def run_command(command):
    try:
        result = subprocess.run(command, shell=True, check=True, capture_output=True, text=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error executing command: {e.stderr}"

def extract_gpu_memory(nvidia_smi_output):
    lines = nvidia_smi_output.split("\n")
    for line in lines:
        if "MiB" in line:
            memory_info = line.split("|")[2].strip()
            used_memory, total_memory = memory_info.split("/")
            total_memory = total_memory.split()[0].strip()
            return int(total_memory)
    return None

def recommend_gpt_size(gpu_memory, user_input):
    if user_input.lower() == "large":
        if gpu_memory >= 8192:
            return f"Based on your GPU's available memory of {gpu_memory} MB, you can use a 'large' GPT model size."
        else:
            return f"Based on your GPU's available memory of {gpu_memory} MB, I recommend using a smaller GPT model size to ensure optimal performance. A 'medium' or 'small' model would be more suitable for your GPU."
    elif user_input.lower() == "medium":
        if gpu_memory >= 4096:
            return f"Based on your GPU's available memory of {gpu_memory} MB, you can use a 'medium' GPT model size."
        else:
            return f"Based on your GPU's available memory of {gpu_memory} MB, I recommend using a 'small' GPT model size for optimal performance."
    else:
        return f"Based on your GPU's available memory of {gpu_memory} MB, a 'small' GPT model size would be the most suitable choice."

def llm_query(query):
    response = agent.run(query)
    return response

def execute_command(command):
    print(f"Executing command: {command}")
    result = run_command(command)
    print(result)

llm = OpenAI(temperature=0)

tools = [
    Tool(
        name="Shell",
        func=run_command,
        description="Run shell commands"
    )
]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

def main():
    while True:
        user_input = input("Enter your query (or 'exit' to quit): ").strip().lower()

        if user_input == 'exit':
            break

        if any(keyword in user_input for keyword in ["gpu", "nvidia", "gpt model size"]):
            print("Executing nvidia-smi command:")
            gpu_info = run_command("nvidia-smi")
            print(gpu_info)
            gpu_memory = extract_gpu_memory(gpu_info)
            if gpu_memory:
                print(f"Total GPU Memory: {gpu_memory} MB")
                if "gpt model size" in user_input:
                    size_input = input("What size GPT model do you want to use? (small, medium, large) ").strip().lower()
                    recommendation = recommend_gpt_size(gpu_memory, size_input)
                    print("Recommendation:", recommendation)
            else:
                print("Unable to determine GPU memory. Please make sure you have an NVIDIA GPU and the nvidia-smi command is available.")
        else:
            # Generate the appropriate command using the LLM
            response = llm_query(f"Generate a Linux shell command to {user_input}. Respond with only the command, no explanations.")

            # Execute the generated command
            execute_command(response)

if __name__ == "__main__":
    main()

Enter your query (or 'exit' to quit): what is ram size


> Entering new AgentExecutor chain...
 I should use the shell command to get the ram size
Action: Shell
Action Input: cat /proc/meminfo | grep MemTotal
Observation: MemTotal:       13290460 kB

Thought: I now know the final answer
Final Answer: cat /proc/meminfo | grep MemTotal

> Finished chain.
Executing command: cat /proc/meminfo | grep MemTotal
MemTotal:       13290460 kB

Enter your query (or 'exit' to quit): which system using more ram


> Entering new AgentExecutor chain...
 I should use the shell command to find the system using more ram
Action: Shell
Action Input: ps aux --sort -rss
Observation: USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root       17875  3.8  1.6 965796 216216 ?       Rl   07:35   0:19 node /datalab/web/pyright/pyright
root        1792  1.7  1.6 1229928 213584 ?      Ssl  06:29   1:20 /usr/bin/python3 -m colab_kernel_
root          95  0.1  1.3 575480 176692 ?       Sl   

KeyboardInterrupt: Interrupted by user

In [22]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            12Gi       1.0Gi       8.9Gi       2.0Mi       2.8Gi        11Gi
Swap:             0B          0B          0B


In [19]:
pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=096d978e89e7831fac8cbf7092e136568c5166155c5931f85e1343bf3c204be2
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
